In [2]:
%pylab inline
np.set_printoptions(precision=4, suppress=True)
from transformations import rotation_matrix
np.__version__

Populating the interactive namespace from numpy and matplotlib


'1.10.1'

In [3]:
gp_table = np.array([1,  2,  3,  4,  5,  6,  7,  8,
                     2,  1,  7, -6,  8, -4,  3,  5,
                     3, -7,  1,  5,  4,  8, -2,  6,
                     4,  6, -5,  1, -3,  2,  8,  7,
                     5,  8, -4,  3, -1, -7,  6, -2,
                     6,  4,  8, -2,  7, -1, -5, -3,
                     7, -3,  2,  8, -6,  5, -1, -4, 
                     8,  5,  6,  7, -2, -3, -4, -1]).reshape(8,8)

def gp_tensor():
    dim = 8
    tensor = np.zeros((8,8,8))
    for k in range(dim):
        for i in range(dim):
            for j in range(dim):
                val = gp_table[i,j]
                if abs(val) == k + 1:
                    tensor[k,i,j] = np.sign(val)
    return tensor

def gp_left_matrix(multivector):
    tensor = gp_tensor()
    matrix = np.zeros((8,8))
    for i in range(8):
        t = tensor[i,:,:]
        matrix[i,:] = np.inner(t.T,multivector.T).reshape(-1)
    return matrix

def gp_right_matrix(multivector):
    tensor = gp_tensor()
    matrix = np.zeros((8,8))
    for i in range(8):
        t = tensor[i,:,:]
        matrix[i,:] = np.inner(multivector.T,t).reshape(-1)
    return matrix

In [23]:
angle = pi/3
R = rotation_matrix(angle, [0,1,0])[:3,:3]
print(R)
np.dot(R, np.array([1,0,0]).reshape(3,1))

[[ 0.5    0.     0.866]
 [ 0.     1.     0.   ]
 [-0.866  0.     0.5  ]]


array([[ 0.5  ],
       [ 0.   ],
       [-0.866]])

In [24]:
A1 = np.array([0.0,         1.0,        0.0, 0.0, 0.0, 0.0, 0.0, 0.0]).reshape(8,1)
A2 = np.array([0.0,         0.0,        0.0, 1.0, 0.0, 0.0, 0.0, 0.0]).reshape(8,1)
B1 = np.array([0.0,  cos(angle), 0.0, -sin(angle), 0.0, 0.0, 0.0, 0.0]).reshape(8,1)
B2 = np.array([0.0, sin(angle), 0.0, cos(angle), 0.0, 0.0, 0.0, 0.0]).reshape(8,1)

In [25]:
A1

array([[ 0.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.]])

In [26]:
gp_a = np.row_stack([gp_right_matrix(A1),gp_right_matrix(A2)])
b_gp = np.row_stack([gp_left_matrix(B1),gp_left_matrix(B2)])
m = gp_a - b_gp
print(m.shape)

(16, 8)


In [27]:
b_gp

array([[ 0.   ,  0.5  ,  0.   , -0.866,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.5  ,  0.   ,  0.   ,  0.   ,  0.   ,  0.866,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   , -0.866,  0.   , -0.5  ,  0.   ],
       [-0.866,  0.   ,  0.   ,  0.   ,  0.   ,  0.5  ,  0.   ,  0.   ],
       [ 0.   ,  0.   , -0.866,  0.   ,  0.   ,  0.   ,  0.   ,  0.5  ],
       [ 0.   ,  0.866,  0.   ,  0.5  ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   , -0.5  ,  0.   ,  0.   ,  0.   ,  0.   , -0.866],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.5  ,  0.   , -0.866,  0.   ],
       [ 0.   ,  0.866,  0.   ,  0.5  ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.866,  0.   ,  0.   ,  0.   ,  0.   , -0.5  ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.5  ,  0.   , -0.866,  0.   ],
       [ 0.5  ,  0.   ,  0.   ,  0.   ,  0.   ,  0.866,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.5  ,  0.   ,  0.   ,  0.   ,  0.   ,  0.866],
       [ 0.   , -0.5  ,  0.   ,  0.866,  0.   ,  0.

In [31]:
[U,s,Vt] = np.linalg.svd(m)
rotor = Vt.T[:,-1]
print(rotor)

[-0.     -0.      0.7071 -0.5     0.      0.5    -0.     -0.    ]


In [29]:
from transformations import quaternion_matrix
R_est = quaternion_matrix([-rotor[4], rotor[5], -rotor[6], rotor[0]])[:3,:3]
print(R_est)

[[-1.  0. -0.]
 [-0.  1.  0.]
 [ 0.  0. -1.]]


In [30]:
np.allclose(R,R_est)

False